In [1]:
!pip install fast_bleu

  Created wheel for fast-bleu: filename=fast_bleu-0.0.90-cp37-cp37m-linux_x86_64.whl size=578940 sha256=ae929c7dbdf78a4be8940dfb923cde20d4db83bc32173ae3d45bf1530c6f3f9d
  Stored in directory: /root/.cache/pip/wheels/bb/68/af/8d3adf7eca3609c20e9c59b27187deb5e7134b52c9df2233d0
Successfully built fast-bleu


In [3]:
from fast_bleu import BLEU
import pandas as pd
import numpy as np
import random

gen_files = ['rnn-baseline.txt', 'rnn-adj.txt', 'seqgan.txt', 'textgan.txt', 'leakgan.txt']

# Setup reference text
ref = ['test.txt']
ref = open('test.txt', 'r').readlines()
n_ref = []
for i in ref:
  i = i.strip('\n').split(' ') # strip newline char and tokenize
  n_ref.append(i[:10]) # limit reference lines to 10 tokens for evaluation to match model output
ref = n_ref

df_scores = pd.DataFrame() # init dataframe for storing scores

# Iterate through the generated text files
for f in gen_files:
  hypotheses = open(f, 'r').readlines()
  formatted_hyp = []
  for i in hypotheses:
    i = i.strip('\n').split(' ') # strip newline char and tokenize
    formatted_hyp.append(i)
  
  # Setup weights to calculate 1-4 n-grams with BLEU
  weights = {'bleu-1': [1.0, 0, 0, 0], 
             'bleu-2': [0, 1.0, 0, 0],
             'bleu-3': [0, 0, 1.0, 0],
             'bleu-4': [0, 0, 0, 1.0],
             }

  # Get BLEU scores
  bleu = BLEU(n_ref, weights)
  results = bleu.get_score(formatted_hyp)

  # Process BLEU scores: average observations for each score and store in a dict
  bleu_dict = {}
  for n_gram in range(1,5):
      bleu_dict[f'bleu-{n_gram}'] = (np.array(results[f'bleu-{n_gram}']).mean())

  # Add model scores to temp dataframe, add a column with the model name, and then append to the main scores dataframe
  tmp_df = pd.DataFrame([bleu_dict])
  tmp_df.insert(0, "Model", [f]) 
  df_scores = df_scores.append(tmp_df, ignore_index=False)

print(df_scores)
df_scores.to_csv('eval_scores.csv') # save eval scores to .csv

              Model    bleu-1    bleu-2    bleu-3    bleu-4
0  rnn-baseline.txt  0.415113  0.073969  0.038026  0.046169
0       rnn-adj.txt  0.744216  0.265993  0.057585  0.044512
0        seqgan.txt  0.785568  0.175680  0.056301  0.062931
0       textgan.txt  0.811072  0.187394  0.045507  0.049414
0       leakgan.txt  0.802498  0.227148  0.045176  0.033724
